In [1]:
import pandas as pd
from pandas.api.types import is_object_dtype
import os
import numpy as np

# Integrity and logical consistency of the data

## Value types and shape

The expected output types are:
- 16 x 3 string columns: 
    - 1 x 3 for LV_P_installed.csv
    - 1 x 3 for LV_generation.csv
    - 1 x 3 for LV_std.csv
    - 1 x 3 for MV_P_installed.csv
    - 1 x 3 for MV_generation.csv
    - 1 x 3 for MV_std.csv
    - 1 x 3 for BESS_allocation_LV.csv
    - 1 x 3 for BESS_allocation_MV.csv
    - 2 x 3 for LV_heat_pump_allocation.csv
    - 2 x 3 for MV_heat_pump_allocation.csv
    - 1 x 3 for EV_allocation.csv
    - 1 x 3 for EV_power_profiles.csv
    - 1 x 3 for LV_basicload_shares.csv.

The other columns are numeric

In [2]:
# we get the name of the files 
folders = os.listdir()
folders = [folder for folder in folders if '.' not in folder] # we remove the files from the list
# files per folder
years = ['2030', '2040', '2050']
files = []
for folder in folders:
    for year in years:
        # we get the name of the files 
        files_list = os.listdir(os.path.join(folder, year))
        files += [os.path.join(folder, year, file) for file in files_list if file.endswith('.csv')]
# we read the files 
dfs = {}
for file in files:
    # we read the file
    df = pd.read_csv(file)
    # we get the columns with object type
    cols = [col for col in df.columns if is_object_dtype(df[col])]
    # we convert the columns to string type
    for col in cols:
        df[col] = df[col].astype('string')
    der, year, type_file = file.split('\\')
    # we add the dataframe to the dictionary
    dfs[(der, year, type_file)] = df

In [3]:
# we get the types of the dataframe
cols, dtypes = [], []
der_list, year_list, type_file_list = [], [], []
for key, df_temp in dfs.items():
    der, year, type_file = key
    # we get the name of the columns
    cols += df_temp.columns.tolist()
    # we get the types of the columns
    dtypes += [str(t) for t in df_temp.dtypes.tolist()]
    # we get the identifiers of the file
    der_list += [der] * len(df_temp.columns)
    year_list += [year] * len(df_temp.columns)
    type_file_list += [type_file] * len(df_temp.columns)
# we create a dataframe with the types of the columns
df_types = pd.DataFrame({'column': cols, 'type': dtypes, 'der': der_list, 'year': year_list, 'type_file': type_file_list})

In [4]:
# we check the non float64 and int64 columns
# the string types must be 16 * 3 
if df_types[~df_types['type'].isin(['float64', 'int64'])].shape[0] == 16 * 3:
    print('The identifiers are correct')
else:
    print('ERROR: The identifiers are not correct')

The identifiers are correct


In [5]:
# we check if the other columns are numeric
if (df_types.shape[0] - df_types[df_types['type'].isin(['float64', 'int64'])].shape[0]) == df_types[~df_types['type'].isin(['float64', 'int64'])].shape[0]:
    print('The numeric columns are correct')
else:
    print('ERROR: The numeric columns are not correct')

The numeric columns are correct


In [6]:
# we print the shape of the dataframe to check if the expected number of columns is correct
der_list, year_list, type_list, rows_list, column_list = [], [], [], [], []
for key, df_temp in dfs.items():
    der, year, type_file = key
    # we get the shape of the dataframe
    der_list.append(der)
    year_list.append(year)
    type_list.append(type_file)
    rows_list.append(df_temp.shape[0])
    column_list.append(df_temp.shape[1])
# we create a dataframe with the shape of the dataframes
df_shape = pd.DataFrame({'der': der_list, 'year': year_list, 'type_file': type_list, 'columns': column_list, 'rows': rows_list})

The correct unique number of columns are (as shown in the Supplementary material): 1, 3, 4, 6, 7, 53, 290, 366, 8761, 8762.

Total unique number of columns: 10

In [8]:
print(sorted(df_shape['columns'].unique())) # we check the number of columns
print('Unique number of columns: ', len(df_shape['columns'].unique())) # we check the number of columns

[1, 3, 4, 6, 7, 53, 290, 366, 8761, 8762]
Unique number of columns:  10


The correct unique number of rows are (as shown in the Supplementary material): 440, 2148, 3625, 6444, 7498, 8760, 11202, 11551, 13726, 13784, 14759, 15429, 19452, 151034, 384288, 481318, 630171, 758118, 876594, 998977, 1065403, 1427096, 2525530

Total unique number of rows: 23

In [9]:
print(sorted(df_shape['rows'].unique()))
print('Unique number of rows: ', len(df_shape['rows'].unique())) # we check the number of rows

[440, 2148, 3625, 6444, 7498, 8760, 11202, 11551, 13726, 13784, 14759, 15429, 19452, 151034, 384288, 481318, 630171, 758118, 876594, 998977, 1065403, 1427096, 2525530]
Unique number of rows:  23


## None values

In [10]:
# we check if there are none values
none_values = False
for key, df_temp in dfs.items():
    der, year, type_file = key
    # we check if there are none values
    if df_temp.isnull().sum().sum() > 0:
        none_values = True
        print(f'ERROR: The file {key} has none values')
if not(none_values):
    print('The files have no none values')

The files have no none values


## Consistency at the micro-level

The PV generation should not be greater than the installed power rating

In [11]:
for year in years:
    # TODO: delete the multiplication by 0.001 when the dataframes are updated
    LV_installed_geq_gen = (dfs['01_PV', year, 'LV_generation.csv'].iloc[:,2:].max(axis=1) * 0.001 <= dfs['01_PV', year, 'LV_P_installed.csv']['P_installed_kW']).all()
    print(f'LV installed PV power is equal or greater than the generation at every node for year {year}: {LV_installed_geq_gen}')
    MV_installed_geq_gen = (dfs['01_PV', year, 'MV_generation.csv'].iloc[:,2:].max(axis=1) * 0.001 <= dfs['01_PV', year, 'MV_P_installed.csv']['P_installed_kW']).all()
    print(f'MV installed PV power is equal or greater than the generation at every node for year {year}: {MV_installed_geq_gen}', '\n')

LV installed PV power is equal or greater than the generation at every node for year 2030: True
MV installed PV power is equal or greater than the generation at every node for year 2030: True 

LV installed PV power is equal or greater than the generation at every node for year 2040: True
MV installed PV power is equal or greater than the generation at every node for year 2040: True 

LV installed PV power is equal or greater than the generation at every node for year 2050: True
MV installed PV power is equal or greater than the generation at every node for year 2050: True 



We check the consistency of the EV profiles.
- We verify if the base charging is equal or lower than the upper charging profile. Also, we verify that the lower charging profile is equal or lower than the base profile.
- We confirm that the flexible energy per day is equal or lower than the sum of the maximum absolute deviation between the lower/uppder profile with the base during the days.

In [12]:
# as the data was rounded, we check that the flexibility is lower or equal to the sum per day + epsilon
epsilon = 5
for year in years:
    upper = dfs['04_EV', year, 'EV_power_profiles_LV.csv'][dfs['04_EV', '2030', 'EV_power_profiles_LV.csv']['Profile_type'] == 'Upper'].iloc[:,2:].values
    base = dfs['04_EV',year, 'EV_power_profiles_LV.csv'][dfs['04_EV', '2030', 'EV_power_profiles_LV.csv']['Profile_type'] == 'Base'].iloc[:,2:].values
    lower = dfs['04_EV', year, 'EV_power_profiles_LV.csv'][dfs['04_EV', '2030', 'EV_power_profiles_LV.csv']['Profile_type'] == 'Lower'].iloc[:,2:].values
    flexibility = dfs['04_EV', year, 'EV_flexible_energy_profiles_LV.csv'].iloc[:,1:].values
    max_deviation_per_hour = np.maximum(np.abs(upper - base), np.abs(lower - base))
    # we get sum per day, considering that the shape is (n_bfs, n_hours) with n_hours = 8760. We have to sum every 24 hours
    max_deviation_per_day = max_deviation_per_hour.reshape(-1, 365, 24)
    sum_per_day = max_deviation_per_day.sum(axis=2)
    # we check that the flexible profiles are equal or lower than the maximum deviation per day
    valid_flexibility = (flexibility <= sum_per_day + epsilon).all()
    print(f'The upper profile is valid for year {year}: {(upper >= base).all()}')
    print(f'The lower profile is valid for year {year}: {(lower <= base).all()}')
    print(f'The flexibility is valid for year {year}: {valid_flexibility}', '\n')

The upper profile is valid for year 2030: True
The lower profile is valid for year 2030: True
The flexibility is valid for year 2030: True 

The upper profile is valid for year 2040: True
The lower profile is valid for year 2040: True
The flexibility is valid for year 2040: True 

The upper profile is valid for year 2050: True
The lower profile is valid for year 2050: True
The flexibility is valid for year 2050: True 

